# Automated Address Lookup and Geocoding



In [1]:
#! pip install --upgrade google-maps-places
#! pip install --upgrade geopandas
#! pip install --upgrade fiona

In [2]:
from google.maps import places_v1
import pandas as pd

In [10]:
client = places_v1.PlacesClient()
input_data = pd.read_csv("./input_data/organizations_with_some_info.csv")
input_data = input_data.fillna('')
organizations = input_data['organization'].tolist()
websites = input_data['website'].tolist()

In [11]:
addresses = []
cities = []
latitudes = []
longitudes = []
types = []
display_names = []
primary_types = []
phone_numbers = []
websites_AI = []
for j in range(len(organizations)):
    request = places_v1.SearchTextRequest(
        text_query = organizations[j] + websites[j] + 'Region of Waterloo',
    )
    response = client.search_text(request=request, 
                                  metadata=[("x-goog-fieldmask", 
                                             "places.displayName,places.addressComponents,places.formattedAddress,places.location,places.types,places.displayName,places.primaryType,places.nationalPhoneNumber,places.websiteUri")])
    if len(response.places) > 0:
        addresses.append(response.places[0].formatted_address)
        longitudes.append(response.places[0].location.longitude)
        latitudes.append(response.places[0].location.latitude)
        types.append(response.places[0].types)
        display_names.append(response.places[0].display_name)
        primary_types.append(response.places[0].primary_type)
        phone_numbers.append(response.places[0].national_phone_number)
        websites_AI.append(response.places[0].website_uri)
        address_details = response.places[0].address_components
        city = "Unknown"
        for i in address_details:
            if i.types[0] == 'locality':
                city = i.long_text
        cities.append(city)
    else:
        addresses.append('NaN')
        longitudes.append('NaN')
        latitudes.append('NaN')
        types.append('NaN')
        display_names.append('NaN')
        primary_types.append('NaN')
        phone_numbers.append('NaN')
        websites_AI.append('NaN')
        cities.append('NaN')
location_data = pd.DataFrame.from_dict({'org' : organizations,
                                        'AI_formatted_name' : display_names,
                                        'AI_types' : types,
                                        'AI_primary_type' : primary_types,
                                        'AI_website' : websites_AI,
                                        'AI_phone' : phone_numbers,
                                        'AI_address' : addresses,
                                        'AI_city' : cities,
                                        'AI_longitude' : longitudes,
                                        'AI_latitude' : latitudes
                                       })
location_data

,org,AI_formatted_name,AI_types,AI_primary_type,AI_website,AI_phone,AI_address,AI_city,AI_longitude,AI_latitude
0,50x30WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Architecture Conservancy Ontario North Waterlo...,"text: ""Architectural Conservancy Of Ontario In...","[point_of_interest, establishment]",,http://www.acontario.ca/,(416) 367-8075,"401 Richmond St W suite 206, Toronto, ON M5V 3...",Toronto,-79.39507,43.647632
2,Interfaith Community Counselling Centre,"text: ""Interfaith Community Counselling Centre...","[point_of_interest, health, establishment]",health,http://www.interfaithcounselling.ca/,(519) 662-3092,"23B Church St, New Hamburg, ON N3A 1J1, Canada",New Hamburg,-80.707601,43.37533
3,Red Maple Senior Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kalihwíyo (Good Message) Market,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
359,BRESLAU AND DISTRICT RECREATION CENTRE,"text: ""Breslau Community Centre""\nlanguage_cod...","[community_center, event_venue, point_of_inter...",community_center,https://www.woolwich.ca/en/living-here/breslau...,(519) 648-9524,"100 Andover Drive, Breslau, ON N0B 1M0, Canada",Breslau,-80.416522,43.471761
360,Brassroots,"text: ""Brass Butterflies""\nlanguage_code: ""en""\n","[event_venue, fitness_center, gym, point_of_in...",,https://brassbutterflies.com/,(519) 577-7653,"60 Bathurst Dr Unit 14-15, Waterloo, ON N2V 2A...",Waterloo,-80.540233,43.508418
361,WATERLOO REGIONAL BLOCK PARENT PROGRAM INC,"text: ""Waterloo Regional Block Parent Program ...","[point_of_interest, establishment]",,https://blockparents.ca/,(519) 745-8410,"425 Hespeler Rd Suite # 413, Cambridge, ON N1R...",Cambridge,-80.323596,43.396215
362,Beechwood West II Homes Association,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
location_data.to_csv('output_data/organizations_with_some_info_supplement.csv', index = False)

In [13]:
input_data['AI_address'] = addresses
input_data['AI_longitude'] = longitudes
input_data['AI_latitude'] = latitudes
input_data['AI_types'] = types
input_data['AI_formatted_name'] = display_names
input_data['AI_primary_type'] = primary_types
input_data['AI_phone'] = phone_numbers
input_data['AI_website'] = websites_AI
input_data['AI_city'] = cities
input_data

,organization,website,phone,AI_address,AI_longitude,AI_latitude,AI_types,AI_formatted_name,AI_primary_type,AI_phone,AI_website,AI_city
0,50x30WR,https://www.50by30wr.ca,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Architecture Conservancy Ontario North Waterlo...,https://aconwr.ca/,,"401 Richmond St W suite 206, Toronto, ON M5V 3...",-79.39507,43.647632,"[point_of_interest, establishment]","text: ""Architectural Conservancy Of Ontario In...",,(416) 367-8075,http://www.acontario.ca/,Toronto
2,Interfaith Community Counselling Centre,https://www.interfaithcounselling.ca,(519) 662-3092,"23B Church St, New Hamburg, ON N3A 1J1, Canada",-80.707601,43.37533,"[point_of_interest, health, establishment]","text: ""Interfaith Community Counselling Centre...",health,(519) 662-3092,http://www.interfaithcounselling.ca/,New Hamburg
3,Red Maple Senior Association,https://rmsa.ca/,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kalihwíyo (Good Message) Market,https://www.facebook.com/people/Kalihw%C3%ADyo...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
359,BRESLAU AND DISTRICT RECREATION CENTRE,http://breslau.org/,,"100 Andover Drive, Breslau, ON N0B 1M0, Canada",-80.416522,43.471761,"[community_center, event_venue, point_of_inter...","text: ""Breslau Community Centre""\nlanguage_cod...",community_center,(519) 648-9524,https://www.woolwich.ca/en/living-here/breslau...,Breslau
360,Brassroots,http://brassroots.ca/,,"60 Bathurst Dr Unit 14-15, Waterloo, ON N2V 2A...",-80.540233,43.508418,"[event_venue, fitness_center, gym, point_of_in...","text: ""Brass Butterflies""\nlanguage_code: ""en""\n",,(519) 577-7653,https://brassbutterflies.com/,Waterloo
361,WATERLOO REGIONAL BLOCK PARENT PROGRAM INC,http://blockparents.ca/,,"425 Hespeler Rd Suite # 413, Cambridge, ON N1R...",-80.323596,43.396215,"[point_of_interest, establishment]","text: ""Waterloo Regional Block Parent Program ...",,(519) 745-8410,https://blockparents.ca/,Cambridge
362,Beechwood West II Homes Association,http://beechwoodwest2.ca/,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
input_data.to_csv('output_data/organizations_with_some_info_augmented.csv', index = False)